# 📌 Upgrades Made to `DGazeDataset` & `DGazeDataModule`

This dataset + datamodule implementation has been upgraded step-by-step to make it **more consistent, faster, and easier to debug/visualize**.  

---

## 🔹 1. Global Normalization
- A new method `compute_stats()` computes **mean/std** of **all eye images and facial features** across the entire dataset (train + val + test).
- Normalization is now **consistent** across all splits (no train/test mismatch).

---

## 🔹 2. Automatic Stats Handling
- If a precomputed `global_stats.pkl` exists → it is **loaded automatically**.  
- If not → stats are **computed once**, then **saved for future runs**.  

---

## 🔹 3. Dataset Caching
- Added `cache_normalized_dataset()` method.
- After dataset creation in `setup()`, the normalized datasets (train/val/test) are **cached into a pickle file**.
- Greatly speeds up re-runs in Colab.

---

## 🔹 4. Automatic Cache Loading
- If a `normalized_dataset.pkl` cache file exists, it is **loaded directly** inside `prepare_data()`.
- `setup()` automatically skips rebuilding datasets if cache is already loaded.

---

## 🔹 5. Dataset Verification
- Added `verify_dataset()` method:
  - Prints mean/std of normalized eye images.  
  - Prints mean/std of normalized facial features.  
- Ensures preprocessing is correct.

---

## 🔹 6. Sample Plotting
- Added `plot_samples(split="train", num_samples=5)` method:
  - Randomly selects samples.  
  - Displays their **eye images**.  
  - Creates a **scatter plot of gaze points** in normalized `[0,1] × [0,1]` space.  

---

## 🔹 7. Lazy Loading
- Instead of preloading everything into memory, the dataset builds an **index of (driver, sequence, frame)**.  
- Frames are loaded **on demand** → saves memory.

---

## 🔹 8. Simplified Usage
Now the full pipeline is super easy:

```python
# Initialize DataModule
dm = DGazeDataModule(
    data_path="driver_data.pkl",
    split_path="splits.pkl",
    stat_path="global_stats.pkl",          # global stats file
    cache_path="normalized_dataset.pkl",   # optional cache file
    batch_size=64
)

# Prepare data
dm.prepare_data()    # loads stats & cache automatically
dm.setup()           # builds datasets if needed

# Verify normalization
dm.verify_dataset()

# Visualize samples
dm.plot_samples(split="train", num_samples=5)


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import lightning as L
import pickle
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

class DGazeDataset(Dataset):
    def __init__(self, driver_data, drivers, sequences,
                 eye_mean, eye_std, feat_mean, feat_std,
                 transform=False):
        """
        Hybrid Dataset:
        - Lazy loads frames (low memory usage).
        - Uses precomputed global normalization stats for eye images & facial features.
        - Normalizes gaze points to [0,1].
        """

        self.driver_data = driver_data
        self.drivers = drivers
        self.sequences = sequences
        self.transform = transform

        # Save stats directly
        self.eye_mean = eye_mean
        self.eye_std = eye_std
        self.feat_mean = feat_mean
        self.feat_std = feat_std

        self.index = []  # list of (driver, seq_key, frame_idx)

        # Pre-build index list
        for driver in drivers:
            data = driver_data[driver]
            for seq in tqdm(sequences, desc=f"Indexing driver {driver}"):
                seq_key = f"seq{seq}"
                if seq_key in data:
                    num_frames = len(data[seq_key]["left_eye"])
                    for frame_idx in range(num_frames):
                        self.index.append((driver, seq_key, frame_idx))

        print(f"Lazy dataset ready! Total samples: {len(self.index)}")

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        driver, seq_key, frame_idx = self.index[idx]
        data_seq = self.driver_data[driver][seq_key]

        # --- Load & normalize left eye ---
        left_eye = data_seq["left_eye"][frame_idx].astype(np.float32)
        left_eye = (left_eye - self.eye_mean) / (self.eye_std + 1e-6)
        left_eye = np.transpose(left_eye, (2, 0, 1))  # (C,H,W)
        left_eye = torch.tensor(left_eye, dtype=torch.float32)

        # --- Load & normalize facial features ---
        headpose = data_seq["headpose_pupil"][frame_idx, 1:].astype(np.float32)
        face_loc = data_seq["face_location"][frame_idx].astype(np.float32)
        facial = np.concatenate((headpose, face_loc))
        facial = (facial - self.feat_mean) / (self.feat_std + 1e-6)
        facial_features = torch.tensor(facial, dtype=torch.float32)

        # --- Load gaze point & normalize to [0,1] ---
        gaze_point = data_seq["gaze_point"][frame_idx, :2].astype(np.float32).copy()
        gaze_point[0] = np.clip(gaze_point[0], 0, 1919) / 1920.0
        gaze_point[1] = np.clip(gaze_point[1], 0, 1079) / 1080.0
        gaze_point = torch.tensor(gaze_point, dtype=torch.float32)

        if self.transform:
            # torchvision transforms can go here
            pass

        return left_eye, facial_features, gaze_point


In [ ]:

class DGazeDataModule(L.LightningDataModule):
    def __init__(self, data_path, split_path, stat_path,
                 batch_size=64, num_workers=4, transform=False,
                 cache_path=None):
        super().__init__()
        self.data_path = data_path
        self.split_path = split_path
        self.stat_path = stat_path
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transform = transform
        self.cache_path = cache_path

        self.driver_data = None
        self.data_split = None
        self.data_stat = None
        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None

    def prepare_data(self):
        # Load driver data and splits
        with open(self.data_path, "rb") as f:
            self.driver_data = pickle.load(f)
        with open(self.split_path, "rb") as f:
            self.data_split = pickle.load(f)

        # Load or compute stats
        if os.path.exists(self.stat_path):
            print(f"Loading precomputed stats from {self.stat_path}")
            with open(self.stat_path, "rb") as f:
                self.data_stat = pickle.load(f)
        else:
            print("Computing global stats (first run)...")
            self.data_stat = self.compute_stats()
            with open(self.stat_path, "wb") as f:
                pickle.dump(self.data_stat, f)
            print(f"Saved stats to {self.stat_path}")

        # Load cached normalized dataset if available
        if self.cache_path and os.path.exists(self.cache_path):
            print(f"Loading cached normalized dataset from {self.cache_path}")
            with open(self.cache_path, "rb") as f:
                cached = pickle.load(f)
            self.train_dataset, self.val_dataset, self.test_dataset = (
                cached["train"], cached["val"], cached["test"]
            )

    def compute_stats(self):
        print("Computing global stats (first run)...")

        all_features = []
        all_pixels = []

        all_drivers = (
            list(self.data_split["drivers_train"])
            + list(self.data_split["drivers_val"])
            + list(self.data_split["drivers_test"])
        )
        all_sequences = (
            list(self.data_split["sequence_train"])
            + list(self.data_split["sequence_val"])
            + list(self.data_split["sequence_test"])
        )

        for driver in tqdm(all_drivers, desc="Scanning drivers"):
            data = self.driver_data[driver]
            for seq in all_sequences:
                seq_key = f"seq{seq}"
                if seq_key not in data:
                    continue

                num_frames = len(data[seq_key]["left_eye"])
                for frame_idx in range(num_frames):
                    eye_img = data[seq_key]["left_eye"][frame_idx].astype(np.float32)
                    all_pixels.append(eye_img.reshape(-1, 3))

                    headpose = data[seq_key]["headpose_pupil"][frame_idx, 1:].astype(np.float32)
                    face_loc = data[seq_key]["face_location"][frame_idx].astype(np.float32)
                    all_features.append(np.concatenate((headpose, face_loc)))

        all_pixels = np.vstack(all_pixels)
        all_features = np.vstack(all_features)

        self.eye_mean = all_pixels.mean(axis=0)
        self.eye_std = all_pixels.std(axis=0)

        self.feat_mean = all_features.mean(axis=0)
        self.feat_std = all_features.std(axis=0)

        stats = {
            "eye_mean": self.eye_mean,
            "eye_std": self.eye_std,
            "feat_mean": self.feat_mean,
            "feat_std": self.feat_std,
        }

        return stats

    def cache_normalized_dataset(self):
        """Save fully constructed datasets into cache (.pkl)."""
        if self.cache_path is None:
            print("No cache path provided. Skipping caching.")
            return

        print(f"Caching normalized datasets to {self.cache_path} ...")
        cache_data = {
            "train": self.train_dataset,
            "val": self.val_dataset,
            "test": self.test_dataset,
        }
        with open(self.cache_path, "wb") as f:
            pickle.dump(cache_data, f)
        print("Datasets cached successfully!")

    def setup(self, stage=None):
        if self.train_dataset and self.val_dataset and self.test_dataset:
            return  # Already loaded from cache

        # Extract stats
        eye_mean = self.data_stat["eye_mean"]
        eye_std = self.data_stat["eye_std"]
        feat_mean = self.data_stat["feat_mean"]
        feat_std = self.data_stat["feat_std"]

        # Build datasets
        self.train_dataset = DGazeDataset(
            self.driver_data, self.data_split["drivers_train"],
            self.data_split["sequence_train"],
            eye_mean, eye_std, feat_mean, feat_std,
            transform=self.transform
        )
        self.val_dataset = DGazeDataset(
            self.driver_data, self.data_split["drivers_val"],
            self.data_split["sequence_val"],
            eye_mean, eye_std, feat_mean, feat_std,
            transform=self.transform
        )
        self.test_dataset = DGazeDataset(
            self.driver_data, self.data_split["drivers_test"],
            self.data_split["sequence_test"],
            eye_mean, eye_std, feat_mean, feat_std,
            transform=self.transform
        )

        if self.cache_path:
            self.cache_normalized_dataset()

    # --- your verification function ---
    def verify_dataset(self, num_batches=5, split="train"):
        if split == "train":
            loader = self.train_dataloader()
        elif split == "val":
            loader = self.val_dataloader()
        elif split == "test":
            loader = self.test_dataloader()
        else:
            raise ValueError("split must be 'train', 'val', or 'test'")

        print(f"Verifying {split} dataset...")
        eye_means, eye_stds, feat_means, feat_stds = [], [], [], []

        for i, (left_eye, facial_features, gaze_point) in enumerate(loader):
            print(f"Batch {i+1}: eye {left_eye.shape}, facial {facial_features.shape}, gaze {gaze_point.shape}")
            assert not torch.isnan(left_eye).any(), "NaN in left_eye"
            assert not torch.isinf(left_eye).any(), "Inf in left_eye"
            assert not torch.isnan(facial_features).any(), "NaN in facial_features"
            assert not torch.isinf(facial_features).any(), "Inf in facial_features"
            assert (gaze_point >= 0).all() and (gaze_point <= 1).all(), "Gaze point out of [0,1] range"

            eye_means.append(left_eye.mean().item())
            eye_stds.append(left_eye.std().item())
            feat_means.append(facial_features.mean().item())
            feat_stds.append(facial_features.std().item())

            if i + 1 >= num_batches:
                break

        print(f"Eye mean ~ {np.mean(eye_means):.4f}, std ~ {np.mean(eye_stds):.4f}")
        print(f"Feat mean ~ {np.mean(feat_means):.4f}, std ~ {np.mean(feat_stds):.4f}")
        print("Verification passed ✅")

    # --- your plotting function ---
    def plot_samples(self, split="train", num_samples=5):
        if split == "train":
            dataset = self.train_dataset
        elif split == "val":
            dataset = self.val_dataset
        elif split == "test":
            dataset = self.test_dataset
        else:
            raise ValueError("split must be 'train', 'val', or 'test'")

        indices = random.sample(range(len(dataset)), num_samples)

        fig, axes = plt.subplots(1, num_samples, figsize=(15, 3))
        gaze_points = []

        for i, idx in enumerate(indices):
            left_eye, _, gaze_point = dataset[idx]
            img = left_eye.permute(1, 2, 0).numpy()
            img = (img - img.min()) / (img.max() - img.min() + 1e-6)
            axes[i].imshow(img)
            axes[i].axis("off")
            axes[i].set_title(f"Sample {i+1}")
            gaze_points.append(gaze_point.numpy())

        plt.show()

        gaze_points = np.array(gaze_points)
        plt.figure(figsize=(5, 5))
        plt.scatter(gaze_points[:, 0], gaze_points[:, 1], c="red", marker="x", s=80)
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.gca().invert_yaxis()
        plt.xlabel("Normalized X")
        plt.ylabel("Normalized Y")
        plt.title(f"Gaze points ({split} set)")
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.show()

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True,
        )